## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
from subprocess import Popen, PIPE
import os
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode

ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 3
NUMBER_OF_ACTIVATED_PARTICIPANTS = 2
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [ipfs_client.add_str(_nickname + 'avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [1_000_000] * NUMBER_OF_PARTICIPANTS

INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(20)
COEF_DOWN = str(5)
TARGET_CLAIM = str(2)

WALLET_ADDRESS = os.getenv('WALLET_ADDRESS')
NODE_URL = 'https://rpc.space-pussy-1.cybernode.ai:443'
NETWORK = 'space-pussy-1'
DISPLAY_TX_EXECUTION = False

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(21)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom19zmt9dlz3wzu6jqgr9409el7nvxw66u8n67ta5424kwcg0qf269s6t03zt'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom16ulvzd0xp9g6wdhnnhz584ed8xzxm5wkmrsznnzk7qsskegkfutq5djtqs'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1ghwwe2nhxjfy7qulgu6zzmghmnjj3rx8xldr8aytghayxlftz6aqxy4c7k'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(25)
PASSPORT_CONTRACT_ADDRESS = 'bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262'

ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'


def execute_bash(bash_command: str, display_data: bool = False) -> [str, str]:
    _output, _error = Popen(bash_command, shell=True, stdout=PIPE, stderr=PIPE).communicate()
    if _error:
        display_data = True
    if display_data:
        print(bash_command)
        if _output:
            try:
                print(json.dumps(json.loads(_output.decode('utf-8')), indent=4, sort_keys=True))
            except json.JSONDecodeError:
                print(_output)
        if _error:
            print(_error)
    return _output.decode('utf-8'), _error.decode('utf-8')

/Users/sergenedashkovsky/Library/Python/3.8/lib/python/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.10.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
claims_list = []
for i in range(NUMBER_OF_PARTICIPANTS):
    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
                  'nickname': NICKNAME_LIST[i],
                  'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], 'cosmos')
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{WALLET_ADDRESS}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    # print(signed_message.signature)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix='cosmos')
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    claims_list.append(claim_item)

claims_df = pd.DataFrame(claims_list)
claims_df.to_csv('claims_ethereum_test_data_without_proof' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,1000000,john16460346760,QmXPcZ4VEWzYv5pY39Enx5qZGpu5Yw7RUdmHFjnYQ9rHge,0x9b600120d5bd1dde9a9b315a8f430763afb7d76f,0xbc4487513ee05cc385b9146422cccf41686acf9cddcc...,bostrom1fakshqfswsqzgyu270x4gt7qpyfnu7mztlytdg,cosmos1fakshqfswsqzgyu270x4gt7qpyfnu7mzgvscn0,jelly busy sniff ready inherit rule milk episo...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x1c82ecb4d605e974983768ded5eca82c01d00a4708a5...,{'pub_key': 'A/313c5SODUd4Lx+I23OB1G9vteki94PB...,eyJwdWJfa2V5IjoiQS8zMTNjNVNPRFVkNEx4K0kyM09CMU...
1,1000000,john16460346761,QmT57xn69N2UkeBY1i27ypdATse4KGo6xwvwjsQr6TUThC,0x32dd3dd14ed13d718774ed999f83a7cb81c3cf5d,0xadf2c2f6319d93ba775129d2448cf89c793bdbf88126...,bostrom19ru2gxtv4cmmfsk7tq3e8ygnypaxl703j5y6k8,cosmos19ru2gxtv4cmmfsk7tq3e8ygnypaxl70338sfgq,creek interest trash charge output embrace mem...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x22d282e2241fd000ff3b4c837c4a63c75619eb389ff5...,{'pub_key': 'AsgRHIcBBYBB0Pn7WFLAYToqFe0g7Lvuq...,eyJwdWJfa2V5IjoiQXNnUkhJY0JCWUJCMFBuN1dGTEFZVG...
2,1000000,john16460346762,QmTdKbjVWdgsxh13RS77df6uSJyLTAeLBFUMGPtwgQLHEW,0x945f32e5bc2ec6a8216a80945bde2947fb6966e8,0xe16b149b132d38c43186b46c593e55df41a6671da633...,bostrom1qmwkmw9zlxlyepqhvumpr6s5568yv9wqpn4e67,cosmos1qmwkmw9zlxlyepqhvumpr6s5568yv9wqzqp2ye,dose prosper tube source reason typical summer...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0xb988a396f5f4ac8b3a7a15d8cb3f54acf3e0ff2b4a84...,{'pub_key': 'AlfxAZLkb/k1PhyAbfjJqFiK4mUE1rOZG...,eyJwdWJfa2V5IjoiQWxmeEFaTGtiL2sxUGh5QWJmakpxRm...


## Instantiate Contracts

In [3]:
def instantiate_contract(init_query: str, contract_code_id: str, contract_label: str, amount: str = '', display_data: bool = False) -> str:
    _init_output, _init_error = execute_bash(
        f'''INIT='{init_query}' \
            && cyber tx wasm instantiate {contract_code_id} "$INIT" --from $WALLET {'--amount ' + amount + 'boot' if amount else ''} --label "{contract_label}" \
            -y --gas 3500000 --broadcast-mode block -o json --chain-id={NETWORK} --node={NODE_URL}''')
    if display_data:
        try:
            print(json.dumps(json.loads(_init_output.decode('utf-8')), indent=4, sort_keys=True))
        except json.JSONDecodeError:
            print(_init_output)
    if _init_error:
        print(_init_error)
    _init_json = json.loads(_init_output)
    return [event['attributes'][0]['value'] for event in _init_json['logs'][0]['events'] if event['type'] == 'instantiate'][0]

### Instantiate SUBGRAPH Contracts

In [4]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph')
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph')
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom19zmt9dlz3wzu6jqgr9409el7nvxw66u8n67ta5424kwcg0qf269s6t03zt
Avatar subgraph contract address: bostrom16ulvzd0xp9g6wdhnnhz584ed8xzxm5wkmrsznnzk7qsskegkfutq5djtqs
Proof subgraph contract address: bostrom1ghwwe2nhxjfy7qulgu6zzmghmnjj3rx8xldr8aytghayxlftz6aqxy4c7k


### Instantiate Passport Contract

In [5]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport')
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra


### Set executor in the Subgraph Contracts

In [6]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_bash(
                f'''REGISTER='{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}' \
                    && CONTRACT="{subgraph_contract_address}" \
                    && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=600000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Gift Contract

In [7]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test gift')
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1xfhtdeernre33xm83wu0eg08433aejyq0ay6h03qatxqrf40k8zqkuhwax


In [8]:
def get_contract_name(contract_address: str) -> str:
    address_dict = {gift_contract_address: 'Gift Contract',
                    passport_contract_address: 'Passport Contract',
                    WALLET_ADDRESS: 'Passport Owner Address',
                    name_subgraph_contract_address: 'Name Subgraph Contract',
                    avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
                    proof_subgraph_contract_address: 'Proof Subgraph Contract'}
    try:
        return address_dict[contract_address]
    except KeyError:
        return contract_address


def get_name_from_cid(ipfs_hash: str, row=None) -> str:
    if row is None:
        return ipfs_hash
    cid_name_dict = {
        row['avatar']: 'Avatar',
        ipfs_client.add_str(row['nickname']): 'Nickname',
        ipfs_client.add_str(row['ethereum_address']): 'Ethereum Address',
        ipfs_client.add_str(row['cosmos_address']): 'Cosmos Address',
        ipfs_client.add_str(WALLET_ADDRESS): 'Passport Owner Address'}
    try:
        return cid_name_dict[ipfs_hash]
    except KeyError:
        return ipfs_hash


def parse_contract_execution_json(contract_execution_json: str, row=None) -> None:
    print('\nEvents')
    if len(json.loads(contract_execution_json)['logs']) == 0:
        print(json.loads(contract_execution_json)['raw_log'])
    for log_item in json.loads(contract_execution_json)['logs']:
        for event_item in log_item['events']:
            print('')
            if event_item['type'] == 'message':
                if len(event_item["attributes"]) == 3:
                    print(f'message from {get_contract_name(event_item["attributes"][-1]["value"])} {event_item["attributes"][1]["value"]} {event_item["attributes"][0]["value"]}')
                else:
                    print(event_item)
            elif event_item['type'] == 'execute':
                print('execute')
                for attr_item in event_item["attributes"]:
                    if attr_item["key"] == '_contract_address':
                        print(f'\texecute contract: {get_contract_name(attr_item["value"])}')
                    else:
                        print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'reply':
                print('reply')
                for attr_item in event_item["attributes"]:
                    if attr_item["key"] == '_contract_address':
                        print(f'\treply contract: {get_contract_name(attr_item["value"])}')
                    else:
                        print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'cyberlink':
                print('cyberlinks')
                for i, attr_item in enumerate(event_item['attributes']):
                    if attr_item['key'] == 'particleFrom':
                        print(f'\t{get_name_from_cid(attr_item["value"], row=row)} -> {get_name_from_cid(event_item["attributes"][i + 1]["value"], row=row)}')
                    elif attr_item['key'] == 'particleTo':
                        pass
                    elif attr_item['key'] == 'neuron':
                        print(f'\tneuron: {get_contract_name(attr_item["value"])}\n')
                    else:
                        print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'coin_received':
                print('coin received')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'coin_spent':
                print('coin spent')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'wasm':
                print('wasm')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'transfer':
                print('transfer')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            else:
                print(event_item)

### Create Merkle Root

In [9]:
root_source_list = [{'address': _item['ethereum_address'],
                     'amount': str(_item['amount'])} for _item in claims_list]+ \
                    [{'address': _item['cosmos_address'],
                      'amount': str(_item['amount'])} for _item in claims_list]

with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [10]:
execute_bash(f'rm {ROOT_FILE_NAME}')
root_output, root_error = execute_bash(
    f'merkle-airdrop-cli generateRoot --file {ROOT_SOURCE_FILE_NAME} >> {ROOT_FILE_NAME} && cat {ROOT_FILE_NAME}')
root = root_output.replace('\n', '')
print(f'Merkle root: {root}')

Merkle root: b006921573f8fcf0e2267d25506f32a9281961973898bdedc10cc6224bd08071


In [11]:
root_register_output, root_register_error = \
    execute_bash(
        f'''REGISTER='{{"register_merkle_root":{{"merkle_root":"{root}"}}}}' \
            && CONTRACT="{gift_contract_address}" \
            && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET --broadcast-mode block -o json -y --chain-id={NETWORK} --node={NODE_URL}''')
parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: b006921573f8fcf0e2267d25506f32a9281961973898bdedc10cc6224bd08071


#### Get Merkle Root form the Gift Contract

In [12]:
print(
    execute_bash(
        f'''QUERY='{{"merkle_root": {{}}}}' \
            && cyber query wasm contract-state smart {gift_contract_address} "$QUERY" -o json --chain-id={NETWORK} --node={NODE_URL}''')[0]
)

{"data":{"merkle_root":"b006921573f8fcf0e2267d25506f32a9281961973898bdedc10cc6224bd08071"}}



In [13]:
def generate_proof(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    proof_output, _ = execute_bash(f'merkle-airdrop-cli generateProofs --file {ROOT_SOURCE_FILE_NAME} --address {claim_row[network + "_address"]} --amount {claim_row.amount}',
                                   display_data=display_data)
    return proof_output.replace('\n','').replace(" ",'').replace("'", '"')

claims_df.loc[:, 'ethereum_proof'] = claims_df.apply(lambda x: generate_proof(x), axis=1)
claims_df.loc[:, 'cosmos_proof'] = claims_df.apply(lambda x: generate_proof(x, network='cosmos'), axis=1)
claims_df.to_csv('claims_ethereum_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,1000000,john16460346760,QmXPcZ4VEWzYv5pY39Enx5qZGpu5Yw7RUdmHFjnYQ9rHge,0x9b600120d5bd1dde9a9b315a8f430763afb7d76f,0xbc4487513ee05cc385b9146422cccf41686acf9cddcc...,bostrom1fakshqfswsqzgyu270x4gt7qpyfnu7mztlytdg,cosmos1fakshqfswsqzgyu270x4gt7qpyfnu7mzgvscn0,jelly busy sniff ready inherit rule milk episo...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x1c82ecb4d605e974983768ded5eca82c01d00a4708a5...,{'pub_key': 'A/313c5SODUd4Lx+I23OB1G9vteki94PB...,eyJwdWJfa2V5IjoiQS8zMTNjNVNPRFVkNEx4K0kyM09CMU...,"[""f696196686b00c9ca342576f75f957df68a6c8400d67...","[""f0d28166f568dc15526bbfe9debe6499576a1d0f31dc..."
1,1000000,john16460346761,QmT57xn69N2UkeBY1i27ypdATse4KGo6xwvwjsQr6TUThC,0x32dd3dd14ed13d718774ed999f83a7cb81c3cf5d,0xadf2c2f6319d93ba775129d2448cf89c793bdbf88126...,bostrom19ru2gxtv4cmmfsk7tq3e8ygnypaxl703j5y6k8,cosmos19ru2gxtv4cmmfsk7tq3e8ygnypaxl70338sfgq,creek interest trash charge output embrace mem...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x22d282e2241fd000ff3b4c837c4a63c75619eb389ff5...,{'pub_key': 'AsgRHIcBBYBB0Pn7WFLAYToqFe0g7Lvuq...,eyJwdWJfa2V5IjoiQXNnUkhJY0JCWUJCMFBuN1dGTEFZVG...,"[""968be20a70aeb28afb93368b61e23a3c371344ce2745...","[""48e34ad2f19327883ce5d582c74b1031cd69268a8c57..."
2,1000000,john16460346762,QmTdKbjVWdgsxh13RS77df6uSJyLTAeLBFUMGPtwgQLHEW,0x945f32e5bc2ec6a8216a80945bde2947fb6966e8,0xe16b149b132d38c43186b46c593e55df41a6671da633...,bostrom1qmwkmw9zlxlyepqhvumpr6s5568yv9wqpn4e67,cosmos1qmwkmw9zlxlyepqhvumpr6s5568yv9wqzqp2ye,dose prosper tube source reason typical summer...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0xb988a396f5f4ac8b3a7a15d8cb3f54acf3e0ff2b4a84...,{'pub_key': 'AlfxAZLkb/k1PhyAbfjJqFiK4mUE1rOZG...,eyJwdWJfa2V5IjoiQWxmeEFaTGtiL2sxUGh5QWJmakpxRm...,"[""ad257d9cfe41d8bb520eca3d52406ad7df991b88ec65...","[""244a668003cdedfaa013060e88c3e05a6bbde48f0f7c..."


### Create Passports

In [14]:
def create_passport(claim_row: pd.Series, display_data: bool = False):
    return execute_bash(
                f'''REGISTER='{{"create_passport":{{"avatar":"{claim_row["avatar"]}", "nickname":"{claim_row["nickname"]}"}}}}' \
                    && CONTRACT="{passport_contract_address}" \
                    && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=600000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

for index, row in tqdm(claims_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        create_passport_json, _ = create_passport(row, display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(create_passport_json, row=row)
    else:
        create_passport(row)

0it [00:00, ?it/s]


Events

cyberlinks
	Passport Owner Address -> Nickname
	Nickname -> Passport Owner Address
	neuron: Name Subgraph Contract

	Passport Owner Address -> Avatar
	Avatar -> Passport Owner Address
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Passport Contract
	reply contract: Passport Contract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	token_id: 40


### Proof (Add) Address to Passports

In [15]:
def proof_address(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_bash(
            f'''REGISTER='{{"proof_address":{{"address":"{claim_row[network + "_address"]}", "nickname":"{claim_row["nickname"]}", "signature":"{claim_row[network + "_message_signature"]}"}}}}' \
                && CONTRACT="{passport_contract_address}" \
                && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=400000 --chain-id={NETWORK} --node={NODE_URL}''',
            display_data=display_data)

for index, row in tqdm(claims_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 1:
        proof_ethereum_address_json, _ = proof_address(row, display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(proof_ethereum_address_json, row=row)

        proof_cosmos_address_json, _ = proof_address(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(proof_cosmos_address_json, row=row)
    else:
        proof_address(row)
        proof_address(row, network='cosmos')

0it [00:00, ?it/s]


Events

cyberlinks
	Passport Owner Address -> Ethereum Address
	Ethereum Address -> Passport Owner Address
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Passport Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16460346761
	address: 0x32dd3dd14ed13d718774ed999f83a7cb81c3cf5d

Events

cyberlinks
	Passport Owner Address -> Cosmos Address
	Cosmos Address -> Passport Owner Address
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Passport Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16460346761
	address: cosmos19ru2gxtv4cmmfsk7tq3e8ygnypaxl70338sfgq


### Claim

In [16]:
def claim(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_bash(
                f'''CLAIM='{{"claim":{{"nickname":"{claim_row['nickname']}", "gift_claiming_address":"{claim_row[network + "_address"]}", "gift_amount":"{claim_row.amount}", "proof":{claim_row[network + "_proof"]}}}}}' \
                    && cyber tx wasm execute {gift_contract_address} "$CLAIM" --from $WALLET -o json --broadcast-mode block -y --gas 400000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)


for index, row in tqdm(claims_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        claim_ethereum_json, _ = claim(row, display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(claim_ethereum_json)

        claim_cosmos_json, _ = claim(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(claim_cosmos_json)
    else:
        claim_ethereum_json_2, _ = claim(row)
        parse_contract_execution_json(claim_ethereum_json_2)

        claim_cosmos_json_2, _ = claim(row, network='cosmos')
        parse_contract_execution_json(claim_cosmos_json_2)

0it [00:00, ?it/s]


Events

coin received
	receiver: Passport Owner Address
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Passport Owner Address
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: 0x9b600120d5bd1dde9a9b315a8f430763afb7d76f
	target: Passport Owner Address
	amount: 20000000

Events

coin received
	receiver: Passport Owner Address
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Passport Owner Address
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: cosmos1fakshqfswsqzgyu270x4gt7qpyfnu7mzgvscn0
	target: Passport Owner Address
	amount: 2000

### Release Gift

In [17]:
def release(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_bash(
                f'''CLAIM='{{"release":{{"gift_address":"{claim_row[network + "_address"]}"}}}}' \
                    && cyber tx wasm execute {gift_contract_address} "$CLAIM" --from $WALLET -o json --broadcast-mode block -y --gas 400000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

for index, row in tqdm(claims_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        release_ethereum_json, _ = release(row, display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(release_ethereum_json)

        release_cosmos_json, _ = release(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        parse_contract_execution_json(release_cosmos_json)
    else:
        release(row)
        release(row, network='cosmos')


0it [00:00, ?it/s]


Events

coin received
	receiver: Passport Owner Address
	amount: 1990000boot

coin spent
	spender: Gift Contract
	amount: 1990000boot

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Passport Owner Address
	sender: Gift Contract
	amount: 1990000boot

wasm
	_contract_address: Gift Contract
	action: release
	address: Passport Owner Address
	gift_address: 0x9b600120d5bd1dde9a9b315a8f430763afb7d76f
	stage: 9
	amount: 1990000

Events

coin received
	receiver: Passport Owner Address
	amount: 1990000boot

coin spent
	spender: Gift Contract
	amount: 1990000boot

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Passport Owner Address
	sender: Gift Contract
	amount: 1990000boot

wasm
	_contract_address: Gift Contract
	action: release
	address: Passport Owner Address
	gift_address: cosmos1fakshqfswsqzgyu270x4gt7

### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft
#### send_nft
#### mint
#### burn
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +